In [3]:
import nltk
import pandas as pd
import io
import re
from random import shuffle
from string import punctuation
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
#nltk.download('punkt')
#nltk.download('stopwords')
#nltk.download('PorterStemmer')

In [5]:
data = pd.read_csv('cleanerstill.csv', sep=";")

/home/ronsholt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
data['diet'].value_counts()

                       24395
mostly anything        16585
anything                6183
strictly anything       5113
mostly vegetarian       3444
mostly other            1007
strictly vegetarian      875
vegetarian               667
strictly other           452
mostly vegan             338
other                    331
strictly vegan           228
vegan                    136
mostly kosher             86
mostly halal              48
strictly kosher           18
strictly halal            18
kosher                    11
halal                     11
Name: diet, dtype: int64

In [4]:
essay4 = [essay for essay in data["essay4"][:20000]]
words = []
all_words = []
essay_list = []
essay_lister = []
stop_words = stopwords.words('english')
porter = PorterStemmer()

for i, essay in enumerate(essay4):
    tmp = []
    tmp_list = []
    if type(essay) != float:
        splt = essay.split()
        words.extend(splt)
        tmp.extend([porter.stem(w) for w in splt if not w in stop_words])
        for w in tmp:
            if "'" in w:
                tmp_list.extend(w.split("'"))
            else:
                tmp_list.append(w)
        essay_list.append((data["diet"][i],[word for word in tmp_list if word]))

for word in words:
    if "'" in word:
        all_words.extend([w for w in word.split("'") if not w in stop_words])
    else:
        all_words.append(word)

In [5]:
clean_words = [w for w in all_words if not w in stop_words]
stemmed_words = [porter.stem(word) for word in clean_words]
freq_words = nltk.FreqDist(w for w in stemmed_words)
word_features = list(freq_words)[:2000]
#print(word_features)

In [6]:
def document_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    return features

featuresets = [(document_features(t), f) for (f, t) in essay_list if t]
shuffle(featuresets)
train_set, test_set = featuresets[100:], featuresets[:100]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [7]:
print(nltk.classify.accuracy(classifier, test_set))

0.33


In [8]:
classifier.show_most_informative_features(10)

Most Informative Features
           contains(luc) = True            halal : mostly =    560.3 : 1.0
         contains(prune) = True           mostly : mostly =    466.9 : 1.0
           contains(pun) = True            halal :        =    421.0 : 1.0
         contains(dalla) = True           strict : mostly =    350.2 : 1.0
         contains(satan) = True            halal : anythi =    341.2 : 1.0
          contains(cube) = True            halal : anythi =    341.2 : 1.0
          contains(topo) = True           strict :        =    338.3 : 1.0
        contains(access) = True           strict :        =    215.3 : 1.0
       contains(darjeel) = True            halal : anythi =    204.7 : 1.0
        contains(hittin) = True            halal : mostly =    201.4 : 1.0
